In [1]:
import QUANTAXIS as QA
import pandas as pd
# import talib
import datetime 
import datetime as dt
from datetime import timedelta

In [2]:
from numba import jit

In [3]:
import tushare as ts
token = "488917d460bca13177c0b510492fc5fc04d0629bc76cc32a4f73fcf0"
ts.set_token(token)
pro = ts.pro_api()

# 获取ts stock_basics

In [4]:
# 字段信息
# https://www.tushare.pro/document/2?doc_id=25
stock_basics = pro.stock_basic(exchange='SSE', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

In [5]:
stock_basics

,ts_code,symbol,name,area,industry,list_date
0,600000.SH,600000,浦发银行,上海,银行,19991110
1,600004.SH,600004,白云机场,广东,机场,20030428
2,600006.SH,600006,东风汽车,湖北,汽车整车,19990727
3,600007.SH,600007,中国国贸,北京,园区开发,19990312
4,600008.SH,600008,首创环保,北京,环境保护,20000427
...,...,...,...,...,...,...
2129,688799.SH,688799,华纳药厂,湖南,化学制药,20210713
2130,688800.SH,688800,瑞可达,江苏,元器件,20210722
2131,688819.SH,688819,天能股份,浙江,电气设备,20210118
2132,688981.SH,688981,中芯国际,上海,半导体,20200716


In [6]:
sse_stock_basics = pro.stock_basic(exchange='SSE', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
# fields='ts_code,symbol,name,area,industry,list_date'

In [7]:
sse_stock_list_date_raw = sse_stock_basics[['ts_code','name','list_date']]

In [8]:
szse_stock_basics = pro.stock_basic(exchange='SZSE', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

In [9]:
szse_stock_list_date_raw = szse_stock_basics[['ts_code','name','list_date']]

In [10]:
# %time szse_stock_list_date_raw.loc[:,'ts_code'].map(lambda x: x.replace('.SZ','')) ##Wall time: 1.96 ms
%time szse_stock_list_date_raw['ts_code'].map(lambda x: x.replace('.SZ','')) 

Wall time: 994 µs


0       000001
1       000002
2       000004
3       000005
4       000006
         ...  
2687    301338
2688    301339
2689    301349
2690    301366
2691    301369
Name: ts_code, Length: 2692, dtype: object

In [11]:
# szse_stock_list_date_raw['ts_code'].map(lambda x: x.split('.')[0])
# szse_stock_list_date_raw['code'] = 
%time szse_stock_list_date_raw['ts_code'].map(lambda x: x.replace('.SZ',''))

Wall time: 983 µs


0       000001
1       000002
2       000004
3       000005
4       000006
         ...  
2687    301338
2688    301339
2689    301349
2690    301366
2691    301369
Name: ts_code, Length: 2692, dtype: object

In [12]:
# szse_stock_list_date_raw['code'] = , this expression will give warning
%time szse_stock_list_date_raw.loc[:,'code'] = szse_stock_list_date_raw['ts_code'].map(lambda x: x.replace('.SZ',''))

Wall time: 5.01 ms


d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
d:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [13]:
%time sse_stock_list_date_raw.loc[:,'code'] = sse_stock_list_date_raw['ts_code'].map(lambda x: x.replace('.SH',''))

Wall time: 2 ms


In [14]:
szse_stock_list_date_raw

,ts_code,name,list_date,code
0,000001.SZ,平安银行,19910403,000001
1,000002.SZ,万科A,19910129,000002
2,000004.SZ,ST国华,19910114,000004
3,000005.SZ,ST星源,19901210,000005
4,000006.SZ,深振业A,19920427,000006
...,...,...,...,...
2687,301338.SZ,凯格精机,20220816,301338
2688,301339.SZ,通行宝,20220909,301339
2689,301349.SZ,信德新材,20220909,301349
2690,301366.SZ,C一博,20220926,301366


In [15]:
# szse_stock_list_date_raw.loc[:,'listDate'] = szse_stock_list_date_raw['list_date'].map(lambda x:dt.datetime.strptime(x,'%Y%m%d').date().strftime('%Y-%m-%d'))

In [16]:
# sse_stock_list_date_raw.loc[:,'listDate'] = sse_stock_list_date_raw['list_date'].map(lambda x:dt.datetime.strptime(x,'%Y%m%d').date().strftime('%Y-%m-%d'))

In [17]:
sse_stock_list_date = sse_stock_list_date_raw[['code','name','list_date']]
szse_stock_list_date = szse_stock_list_date_raw[['code','name','list_date']]

In [18]:
%time cn_stock_list_date_raw = pd.concat([sse_stock_list_date,szse_stock_list_date])

Wall time: 1 ms


In [19]:
stk_code_name_map = cn_stock_list_date_raw[['code','name']].set_index('code')
stk_code_name_map.head(4)

,name
code,
600000,浦发银行
600004,白云机场
600006,东风汽车
600007,中国国贸


In [20]:
cn_stock_list_date = pd.concat([sse_stock_list_date,szse_stock_list_date],axis=0)

In [21]:
cn_stock_list_date

,code,name,list_date
0,600000,浦发银行,19991110
1,600004,白云机场,20030428
2,600006,东风汽车,19990727
3,600007,中国国贸,19990312
4,600008,首创环保,20000427
...,...,...,...
2687,301338,凯格精机,20220816
2688,301339,通行宝,20220909
2689,301349,信德新材,20220909
2690,301366,C一博,20220926


In [22]:
# szse_stock_list_date_raw['code']= 
# %time szse_stock_list_date_raw['ts_code'].map(lambda x: x.split('.')[0])
# Wall time: 1 ms

In [23]:
# apply 要比map 慢
# %time szse_stock_list_date_raw.apply(lambda x: x.ts_code.split('.')[0],axis=1)
# Wall time: 31 ms

In [24]:
# szse_stock_list_date_raw['code']= szse_stock_list_date_raw.apply(lambda x: x.ts_code.split('.')[0],axis=1)

# 选妖股及对应板块

In [24]:
today = datetime.date.today() - timedelta(days=1)

In [25]:
from QUANTAXIS.QAUtil import trade_date_sse

In [26]:
# 获取所有股票列表
cn_stk_list = QA.QA_fetch_stock_list()
# 排除ST和退市股
cn_stk_list = cn_stk_list[~cn_stk_list['name'].str.contains('ST|退')]

In [27]:
code_list = cn_stk_list.code.to_list()

In [28]:
new_trade_date = [x for x in filter(lambda x:dt.datetime.strptime(x,'%Y-%m-%d').date()<=today,trade_date_sse)]

In [29]:
start_date = new_trade_date[-10]
end_date = new_trade_date[-1]

In [30]:
end_date

'2022-09-27'

In [31]:
# start_time = datetime.datetime.now()
# stk_day_all.apply(lambda x:cacl_base_data(x))
%time  stk_day_all = QA.QA_fetch_stock_day_adv(code_list,start_date,end_date).to_qfq()
# end_time = datetime.datetime.now()
# time_diff = end_time - start_time
# print(time_diff)

Wall time: 11 s


In [32]:
#多日涨幅
def drzf(data):
    return pd.DataFrame(zip(data.groupby(level=1)['close'].pct_change(1),\
                            data.groupby(level=1)['close'].pct_change(2),
                           data.groupby(level=1)['close'].pct_change(3),
                           data.groupby(level=1)['close'].pct_change(4),
                           data.groupby(level=1)['close'].pct_change(5),
                           data.groupby(level=1)['close'].pct_change(6),
                           data.groupby(level=1)['close'].pct_change(7),
                           ),index=data.index)

In [33]:
%time stk_all_pct_chg  = stk_day_all.apply(lambda x:drzf(x))

Wall time: 837 ms


In [34]:
today_stk_pct_chg = stk_all_pct_chg.xs(end_date)

In [35]:
# 今天所有股票从一天到7天的涨跌幅
today_stk_pct_chg

,0,1,2,3,4,5,6
code,,,,,,,
000001,0.012500,-0.011391,-0.011391,-0.022526,-0.015397,-0.033413,-0.032643
000002,0.009174,0.005140,0.006289,0.002278,0.002278,-0.031903,-0.022222
000006,0.014151,-0.013761,-0.009217,-0.031532,-0.002320,-0.002320,0.000000
000007,0.028758,-0.006313,-0.006313,0.000000,0.007682,0.007682,-0.007566
000008,0.027027,0.000000,-0.008696,-0.025641,-0.021459,0.000000,-0.021459
...,...,...,...,...,...,...,...
688799,0.028197,0.017608,-0.009699,-0.030082,-0.019212,-0.012254,-0.043709
688800,-0.003235,0.022631,-0.025729,-0.006304,-0.031714,-0.055199,-0.033302
688819,0.018745,0.038205,-0.001114,0.001676,-0.019410,0.010707,-0.006096


In [36]:
cn_stock_list_date = cn_stock_list_date_raw.set_index('code')

In [37]:
cn_stock_list_date

,name,list_date
code,,
600000,浦发银行,19991110
600004,白云机场,20030428
600006,东风汽车,19990727
600007,中国国贸,19990312
600008,首创环保,20000427
...,...,...
301338,凯格精机,20220816
301339,通行宝,20220909
301349,信德新材,20220909


In [38]:
cn_stock_list_date.loc[:,('list_days')] = cn_stock_list_date['list_date'].map(lambda x:  dt.datetime.strptime(end_date,'%Y-%m-%d').date() - dt.datetime.strptime(str(x),'%Y%m%d').date())

# 获得上市时间少于30天的股票的涨跌幅

In [39]:
# 上市时间少于30天的股票
new_stk_lt_30_d = cn_stock_list_date[cn_stock_list_date.list_days <= timedelta(days=30)]

In [40]:
# 上市时间大于30天的股票
%time old_stk_gt_30_d = cn_stock_list_date[cn_stock_list_date.list_days >= timedelta(days=30)]

Wall time: 1.98 ms


In [42]:
# 小于30天上市股票的涨跌幅
stk_lt_30d_pct_chg = pd.concat([today_stk_pct_chg,new_stk_lt_30_d],axis =1,join='inner')

In [43]:
# stk_pct_chg_list_date.loc[:,('day_diff')] = 
# stk_pct_chg_list_date['list_date'].map(lambda x: dt.datetime.strptime(end_date,'%Y-%m-%d').date())
# stk_pct_chg_list_date.loc[:,('day_diff')] = stk_pct_chg_list_date['list_date'].map(lambda x:  dt.datetime.strptime(end_date,'%Y-%m-%d').date() - dt.datetime.strptime(str(x),'%Y%m%d').date())
top_stk_lt_30d_pct_chg = stk_lt_30d_pct_chg.filter([0,1,2,3,4,5,6]).max(axis=1).sort_values(ascending = False)

In [44]:
stk_lt_30d_pct_chg.loc[:,'max_zf'] =stk_lt_30d_pct_chg.filter([0,1,2,3,4,5,6]).max(axis=1).sort_values(ascending = False)

## 新股涨跌幅

In [45]:
stk_lt_30d_pct_chg.sort_values(by='max_zf',ascending=False)

,0,1,2,3,4,5,6,name,list_date,list_days,max_zf
code,,,,,,,,,,,
001238,0.099946,0.209811,0.330949,0.464235,0.610543,0.771527,NaN,浙江正特,20220919,8 days,0.771527
301331,0.162014,0.218330,0.332459,0.161217,NaN,NaN,NaN,C恩威,20220921,6 days,0.332459
001259,-0.073403,-0.059770,-0.130342,-0.043275,0.052496,0.157656,0.273350,利仁科技,20220831,27 days,0.273350
688114,0.199981,0.264784,0.206562,0.189093,0.194164,0.218738,0.027868,华大智造,20220909,18 days,0.264784
001332,0.014827,0.013719,0.115118,0.226678,0.132511,NaN,NaN,锡装股份,20220920,7 days,0.226678
301327,0.021876,0.028742,0.068919,0.146377,0.024700,0.127316,NaN,C华宝,20220919,8 days,0.146377
688035,0.033563,0.081081,0.130395,0.098507,-0.076421,-0.029664,NaN,C德邦,20220919,8 days,0.130395
603057,0.099908,NaN,NaN,NaN,NaN,NaN,NaN,紫燕食品,20220926,1 days,0.099908
688391,0.040330,0.066926,-0.001856,-0.050784,-0.117572,-0.038212,-0.092313,钜泉科技,20220913,14 days,0.066926


# 非新股涨跌幅榜

## 7日涨跌幅

In [46]:
# stk_all_pct_chg_list_date = pd.concat([stk_all_pct_chg.xs(end_date),cn_stock_list_date],axis =1) 
# concat 是基于index 保留上市超过30天的
stk_gt_30d_pct_chg = pd.concat([today_stk_pct_chg,old_stk_gt_30_d],axis =1,join='inner')

In [49]:
stk_gt_30d_pct_chg.loc[:,'max_zf'] = stk_gt_30d_pct_chg.filter([0,1,2,3,4,5,6]).max(axis=1)

In [53]:
stk_gt_30d_pct_chg.sort_values(by='max_zf',ascending=False)

,0,1,2,3,4,5,6,name,list_date,list_days,max_zf
code,,,,,,,,,,,
003027,0.100111,0.136782,0.250316,0.375522,0.513004,0.664049,0.830352,同兴环保,20201218,648 days,0.830352
003023,0.052632,0.157895,0.107383,0.218274,0.340102,0.405751,0.349693,彩虹集团,20201211,655 days,0.405751
002543,0.100231,0.210348,0.313996,0.283273,0.333645,0.394917,0.382752,万和电气,20110128,4260 days,0.394917
002514,0.065297,0.120370,0.232598,0.194079,0.314027,0.355742,0.380228,宝馨科技,20101203,4316 days,0.380228
002641,0.100186,0.210204,0.332584,0.309051,0.329596,0.360092,0.326622,公元股份,20111208,3946 days,0.360092
...,...,...,...,...,...,...,...,...,...,...,...
002532,-0.058008,-0.092345,-0.132404,-0.133411,-0.134415,-0.139401,-0.108592,天山铝业,20101231,4288 days,-0.058008
300446,-0.063717,-0.102629,-0.121992,-0.121262,-0.109428,-0.065371,-0.071930,乐凯新材,20150423,2714 days,-0.063717
002896,-0.066884,-0.066884,-0.160305,-0.205776,-0.224722,-0.178161,-0.128315,中大力德,20170829,1855 days,-0.066884


In [47]:
stck_gt_30d_7_d_highest_list = stck_gt_30d_7_d_highest.index.to_list()

NameError: name 'stck_gt_30d_7_d_highest' is not defined

In [71]:
# stock_block_data = QA.QA_fetch_stock_block('000001')
def get_industry(code):
    stock_block_data = QA.QA_fetch_stock_block(code)
    return pd.DataFrame(stock_block_data.loc[stock_block_data['type'] == 'tdxhy'].blockname)

In [ ]:
stck_gt_30d_7_d_highest

In [74]:
blk_of_hot_stock =get_industry(stck_gt_30d_7_d_highest_list[0])
for n in stck_gt_30d_7_d_highest_list[1:]:
    blk_of_hot_stock = pd.concat([blk_of_hot_stock,get_industry(n)],axis=0)
    
pd_top30_stock = pd.DataFrame(blk_of_hot_stock)

In [75]:
blk_of_hot_stk = pd.concat([pd_top30_stock,blk_of_hot_stock],axis=1)

In [76]:
blk_of_hot_stk

,blockname,blockname
code,,
003027,环境保护,环境保护
002713,装修装饰,装修装饰
600766,黄金,黄金
003023,家用电器,家用电器
002514,机械基件,机械基件
000620,全国地产,全国地产
603701,汽车配件,汽车配件
002568,红黄酒,红黄酒
002866,IT设备,IT设备


In [231]:
# 妖股所属板块
blk_of_hot_stk.reset_index().filter(['code','blockname']).groupby(['blockname'],as_index=True).count().rename(columns = {'code':'7d_zf_count'}).sort_values(by ='7d_zf_count',ascending = False)

,7d_zf_count
blockname,
环境保护,4
化工原料,3
电气设备,3
专用机械,2
元器件,2
影视音像,2
文教休闲,2
电器仪表,1
铁路,1


# 3日内涨幅榜

In [57]:
stck_all_3_d_highest = stk_all_pct_chg.xs(end_date)[[3]].sort_values(by=3,ascending = False).head(30)

In [60]:
stck_all_3_d_highest_list = stck_all_3_d_highest.index.to_list()

In [61]:
blk_of_hot_stock_3d =get_industry(stck_all_3_d_highest_list[0])
for n in stck_all_3_d_highest_list[1:]:
    blk_of_hot_stock_3d = pd.concat([blk_of_hot_stock_3d,get_industry(n)],axis=0)
    
pd_top30_stock_3d = pd.DataFrame(stck_all_3_d_highest)
pd.concat([pd_top30_stock_3d,blk_of_hot_stock_3d],axis=1)

,3,blockname
code,,
000045,0.464824,元器件
002334,0.332046,电气设备
002469,0.331343,化工原料
002445,0.330579,钢加工
002689,0.330508,运输设备
001236,0.330153,多元金融
300566,0.323154,元器件
300916,0.294577,元器件
688259,0.288466,半导体


In [62]:
blk_of_hot_stock_3d.reset_index().groupby(['blockname'],as_index=True).count().rename(columns = {'code':'count'}).sort_values(by ='count',ascending = False)

,count
blockname,
元器件,4
化工原料,4
电气设备,3
半导体,2
文教休闲,2
机械基件,2
IT设备,1
水力发电,1
运输设备,1


In [ ]:
'QA_fetch_stock_full',
 'QA_fetch_stock_info',
 'QA_fetch_stock_list',

In [66]:
full_day = QA.QA_fetch_stock_full(end_date)

array(['000002', '17.02', '17.15', '16.85', '17.05', '504151.0',
       '2022-08-22'], dtype='<U15')

In [77]:
%timeit stk_info = QA.QA_fetch_stock_info(['000001','000002'])

2.62 ms ± 28.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
stk_info

,market,code,liutongguben,province,industry,updated_date,ipo_date,zongguben,guojiagu,faqirenfarengu,...,touzishouyu,jingyingxianjinliu,zongxianjinliu,cunhuo,lirunzonghe,shuihoulirun,jinglirun,weifenpeilirun,meigujingzichan,baoliu2
code,,,,,,,,,,,,,,,,,,,,,
000001,0,000001,1.940576e+10,18,1,20210203,19910403,1.940592e+10,1.801990e+09,2.819500e+11,...,9.921000e+10,-1.616100e+11,1.031840e+12,0.0,3.675400e+11,2.892800e+11,2.892800e+11,1.311860e+12,15.16,12.0


In [63]:
dir(QA)

['ABS',
 'ADX_MA',
 'AMOUNT_MODEL',
 'ATR_RSI_Stops',
 'ATR_SuperTrend_cross',
 'AVEDEV',
 'BARLAST',
 'BARLAST_EXIST',
 'BBANDS',
 'BBI',
 'BBIBOLL',
 'BETA',
 'BROKER_EVENT',
 'BROKER_TYPE',
 'CMO',
 'CORREL',
 'COUNT',
 'CROSS',
 'CROSS_STATUS',
 'DATABASE',
 'DATASOURCE',
 'DEMA',
 'DIFF',
 'DataApi',
 'DataFrame',
 'EMA',
 'ENGINE_EVENT',
 'EVERY',
 'EXCHANGE_ID',
 'EXIST',
 'FILTER',
 'FREQUENCE',
 'FactorAnalyzer',
 'HHV',
 'HT_DCPERIOD',
 'HT_DCPHASE',
 'HT_PHASOR',
 'HT_SINE',
 'HT_TRENDLINE',
 'HT_TRENDMODE',
 'IF',
 'IFAND',
 'IFOR',
 'KAMA',
 'LAST',
 'LINEARREG',
 'LINEARREG_ANGLE',
 'LINEARREG_INTERCEPT',
 'LINEARREG_SLOPE',
 'LLV',
 'MA',
 'MACD',
 'MACDEXT',
 'MACDFIX',
 'MAMA',
 'MARKET_ERROR',
 'MARKET_EVENT',
 'MARKET_TYPE',
 'MAX',
 'MIN',
 'ORDER_DIRECTION',
 'ORDER_EVENT',
 'ORDER_MODEL',
 'ORDER_STATUS',
 'OUTPUT_FORMAT',
 'QAARP',
 'QAAnalysis',
 'QAAnalysis_Series_slope',
 'QAAnalysis_block',
 'QAAnalysis_dataframe',
 'QAAnalysis_series',
 'QAAnalysis_stock',
 

In [3]:
# 写个指标 so easy
def ifup20(data):
    # QA内建指标计算 Python原生代码
    return (QA.MA(data.close, 5)-QA.MA(data.close, 20)).dropna() > 0

def ifup20_TA(data):
    # TA-lib计算
    return (TA_MA(data.close, 5)-TA_MA(data.close, 20)).dropna() > 0

In [4]:
idx_list = QA.QA_fetch_index_list_adv()

In [5]:
idx_code = idx_list['880301':].code
industry_code = idx_code['880301':'880497'].to_list()  #132  行业指数代码
concept_code = idx_code['880501':'880981'].to_list() #362   概念指数代码
idx_name = idx_list['880301':'880981'].name  #所有板块指数名称
#所有行业指数名
industry_name = idx_name['880301':'880497'] 
#所有概念指数名
concept_name = idx_name['880501':'880981']

In [6]:
industry_data = QA.QA_fetch_index_day_adv(industry_code,'2022-07-15','2022-07-15')

In [7]:
idx_name

code
880301        煤炭
880302      煤炭开采
880303      焦炭加工
880305        电力
880306      水力发电
           ...  
880967      数字货币
880968      胎压监测
880969      无线耳机
880970      分拆预期
880981    TDX 能源
Name: name, Length: 494, dtype: object

In [8]:
all_industry_data = industry_data.data.droplevel(level=0).sort_values('amount',ascending=False)

In [9]:
pd.concat([all_industry_data,industry_name],axis=1)

,open,close,high,low,vol,amount,up_count,down_count,date_stamp,volume,name
code,,,,,,,,,,,
880446,2365.06,2352.00,2407.93,2352.00,700175.0,1.488008e+11,89,160,1.657814e+09,700175.0,电气设备
880390,2292.51,2317.70,2367.86,2291.32,591293.0,8.698075e+10,111,120,1.657814e+09,591293.0,汽车类
880335,2145.35,2123.20,2168.74,2123.20,466380.0,8.010907e+10,98,269,1.657814e+09,466380.0,化工
880324,986.87,976.66,1002.34,976.65,416931.0,6.177614e+10,31,96,1.657814e+09,416931.0,有色
880400,2235.55,2192.64,2250.71,2192.64,325770.0,5.705162e+10,34,235,1.657814e+09,325770.0,医药
...,...,...,...,...,...,...,...,...,...,...,...
880313,1073.14,1056.39,1081.92,1053.70,6025.0,4.467032e+08,0,7,1.657814e+09,6025.0,石油贸易
880412,626.59,612.08,627.51,611.91,8766.0,4.401384e+08,0,4,1.657814e+09,8766.0,商品城
880453,952.10,934.13,954.81,930.65,6381.0,4.010337e+08,1,5,1.657814e+09,6381.0,公共交通


概念指数

In [10]:
concept_data = QA.QA_fetch_index_day_adv(concept_code,'2022-07-15','2022-07-15') 

In [11]:
all_concept_data = concept_data.data.droplevel(level=0).sort_values('amount',ascending=False)

In [12]:
pd.concat([all_concept_data,concept_name],axis=1)

,open,close,high,low,vol,amount,up_count,down_count,date_stamp,volume,name
code,,,,,,,,,,,
880821,1663.32,1645.66,1680.63,1645.66,1224093.0,2.788157e+11,34.0,164.0,1.657814e+09,1224093.0,大盘股
880883,1386.96,1370.52,1400.09,1370.52,1664490.0,2.762124e+11,44.0,267.0,1.657814e+09,1664490.0,MSCI成份
880534,2901.82,2873.34,2950.59,2873.34,910162.0,1.934394e+11,90.0,212.0,1.657814e+09,910162.0,锂电池
880801,2154.21,2134.02,2184.08,2134.02,451154.0,1.917428e+11,21.0,79.0,1.657814e+09,451154.0,基金重仓
880951,1623.02,1617.27,1658.68,1617.27,977659.0,1.868514e+11,151.0,226.0,1.657814e+09,977659.0,新能源车
...,...,...,...,...,...,...,...,...,...,...,...
880777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,罗素小盘
880925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,特钢
880928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,抗流感


In [13]:
# hs300 = blk.get_block('沪深300').code

In [14]:
today = datetime.date.today()

In [15]:
from QUANTAXIS.QAUtil import trade_date_sse
import datetime as dt

In [17]:
new_trade_date = [x for x in filter(lambda x:dt.datetime.strptime(x,'%Y-%m-%d').date()<=today,trade_date_sse)]

In [19]:
new_trade_date[-2]

'2022-08-19'

In [83]:
# data = QA.QA_fetch_stock_day_adv(hs300,'2020-01-01','2020-09-14')
# data = QA.QA_fetch_stock_day_adv('000001','2021-01-01','2021-02-01')

In [20]:
# 获取所有股票列表
cn_stk_list = QA.QA_fetch_stock_list()
# 排除ST和退市股
cn_stk_list = cn_stk_list[~cn_stk_list['name'].str.contains('ST|退')]
# 获取20天交易日
#     today= datetime.date.today()
# last_working_day = datetime.date.today()
# delta = datetime.timedelta(days=3)
# start = (today - delta).strftime('%Y-%m-%d')
# end = today.strftime('%Y-%m-%d')
code_list = cn_stk_list.code.to_list()


In [28]:
end_date =new_trade_date[-30]

In [21]:
stk_day_all = QA.QA_fetch_stock_day_adv(code_list,new_trade_date[-41],new_trade_date[-30]).to_qfq()

In [16]:
stk_day_all2 = QA.QA_fetch_stock_day_adv(code_list,'2022-07-01','2022-07-14').to_qfq()

In [151]:
# stk_day_all3  = QA.QA_fetch_stock_day_adv(code_list,'2022-07-01','2022-08-19')

In [145]:
# stk_day_all4 = QA.QA_fetch_stock_day_adv(code_list,new_trade_date[-11],new_trade_date[-1])

In [23]:
stk_day_all.data
# 'DataFrame' object has no attribute 'date'

open        high         low       close     volume  \
date       code                                                                
2022-06-27 000001   13.990444   14.304836   13.970795   14.236063  1497278.0   
           000002   18.360000   18.660000   18.280000   18.510000   700741.0   
           000006    4.500000    4.680000    4.440000    4.610000   468974.0   
           000007    6.970000    6.980000    6.870000    6.940000    37821.0   
           000008    2.490000    2.530000    2.480000    2.490000   172336.0   
...                       ...         ...         ...         ...        ...   
2022-07-12 688799   30.750000   30.760000   29.730000   29.830000     3805.0   
           688800  135.660000  141.680000  133.660000  136.480000     7338.0   
           688819   34.580000   34.880000   33.990000   34.020000    18229.0   
           688981   41.880000   42.240000   41.880000   41.880000    97544.0   
           689009   43.600000   44.300000   41.710000   41.960000    27164.0   

                         amount       adj  
date       code                            
2022-06-27 000001  2.166381e+09  0.982475  
           000002  1.293838e+09  1.000000  
           000006  2.157314e+08  1.000000  
           000007  2.617541e+07  1.000000  
           000008  4.305568e+07  1.000000  
...                         ...       ...  
2022-07-12 688799  1.138110e+07  1.000000  
           688800  1.005256e+08  1.000000  
           688819  6.261976e+07  1.000000  
           688981  4.102674e+08  1.000000  
           689009  1.163586e+08  1.000000  

[54790 rows x 7 columns]

In [24]:
#多日涨幅
def drzf(data):
    return pd.DataFrame(zip(data.groupby(level=1)['close'].pct_change(1),\
                            data.groupby(level=1)['close'].pct_change(2),
                           data.groupby(level=1)['close'].pct_change(3),
                           data.groupby(level=1)['close'].pct_change(4),
                           data.groupby(level=1)['close'].pct_change(5),
                           data.groupby(level=1)['close'].pct_change(6),
                           data.groupby(level=1)['close'].pct_change(7),
                           ),index=data.index)

In [25]:
%time grp_method = stk_day_all.apply(lambda x:drzf(x))

Wall time: 200 ms


In [29]:
# grp_method.xs('2022-08-19').max(axis=1).sort_values(ascending = False).dropna().head(20) 
grp_method.xs(end_date).max(axis=1).sort_values(ascending = False).dropna().head(20)

code
002090    0.949438
300565    0.701613
002589    0.624352
000755    0.611794
002204    0.594595
000629    0.555024
002992    0.505732
002333    0.492395
002514    0.465923
002965    0.453685
688218    0.444611
301268    0.438515
301091    0.426080
688607    0.405481
002219    0.405405
000404    0.400749
688290    0.395250
600192    0.372277
000722    0.358491
000600    0.346983
dtype: float64

In [30]:
top20_stock = grp_method.xs(end_date).max(axis=1).sort_values(ascending = False).dropna().head(20)

In [31]:
top20_code_list = top20_stock.index.to_list()

In [32]:
# stock_block_data = QA.QA_fetch_stock_block('000001')
def get_industry(code):
    stock_block_data = QA.QA_fetch_stock_block(code)
    return pd.DataFrame(stock_block_data.loc[stock_block_data['type'] == 'tdxhy'].blockname)

In [36]:
df=get_industry(top20_code_list[0])
for n in top20_code_list[1:]:
    df = pd.concat([df,get_industry(n)],axis=0)
    
pd_top20_stock = pd.DataFrame(top20_stock)
pd.concat([pd_top20_stock,df],axis=1)

In [42]:
df.sort_values(by = 'blockname',axis = 0)

,blockname
code,
688290,专用机械
002204,专用机械
688218,专用机械
002992,元器件
000722,其他建材
002219,医疗保健
688607,医疗保健
002589,医药商业
000404,家用电器


In [44]:
pd.concat([pd_top20_stock,df],axis=1)

,0,blockname
code,,
002090,0.949438,软件服务
300565,0.701613,通信设备
002589,0.624352,医药商业
000755,0.611794,路桥
002204,0.594595,专用机械
000629,0.555024,小金属
002992,0.505732,元器件
002333,0.492395,铝
002514,0.465923,机械基件


In [157]:
import tushare as ts

In [159]:
# 排除新股
# stock_basics = ts.get_stock_basics()
# time_to_market = stock_basics.ix[code]['TimeToMarket']

'SZ'

# 计算基础数据

In [214]:
def cacl_base_data(data):
    temp = {
        'open':data.open,
        'high':data.high,
        'low': data.low,
        'close': data.close,
        'volume': data.volume,
        'amount':data.amount
    }
    #多日涨幅  统计过去1，2，3，4，5，6，7，8，9天的涨幅
    drzf = {'zf{}'.format(i): data.close.pct_change(i)*100 for i in range(10)[1:10]}
    temp.update(drzf)
    
    #多日emas
    emas = {'ema{}'.format(n): QA.EMA(data.close,n) for n in [5,10,15,20,30,40,60,80,100,120,240]}
    temp.update(emas)
    
    #多日波动幅度
    mazf = {'mazf{}'.format(n): QA.MA(data.close.pct_change(1)*100,n) for n in [5,10,15,20,30,40,60,80,100,120,240]}
    temp.update(mazf)
    
    #量能均值
    mavol = {'mavol{}'.format(n): QA.MA(data.volume,n) for n in [10,20,60,120,240]}
    temp.update(mavol)
    
    #成交额均线
    maamo = {'maamo{}'.format(n): QA.MA(data.amount,n) for n in [10,20,60,120,240]}
    temp.update(maamo)
    
    return pd.DataFrame(temp)    

In [215]:
start_time = datetime.datetime.now()
stk_day_all.apply(lambda x:cacl_base_data(x))
end_time = datetime.datetime.now()
time_diff = end_time - start_time

0:00:00.640221


# 选择妖股

In [124]:
def f(data):
#     return pd.DataFrame({'zf{}'.format(i): data.close.pct_change(i)*100 for i in range(10)[1:10]})
    return pd.DataFrame({'zf1': data.close.pct_change(1)*100,'zf2': data.close.pct_change(2)*100,'zf3': data.close.pct_change(3)*100})

In [ ]:
ind_base_data = stk_day_all.add_func(f) 

In [125]:
# start_time = datetime.datetime.now()
ind_base_data = stk_day_all.add_func(f)  #84sec
# stk_day_all.apply(lambda x:f(x))
#涨幅最大的20只
tt = ind_base_data.loc['2022-08-19'].max(axis=1).sort_values(ascending=False).head(20)
end_time = datetime.datetime.now()
print(end_time - start_time)
#0:00:12.371899

0:00:14.373023


In [127]:
ind_base_data

zf1       zf2       zf3
date       code                                
2022-08-16 000001       NaN       NaN       NaN
           000002       NaN       NaN       NaN
           000006       NaN       NaN       NaN
           000007       NaN       NaN       NaN
           000008       NaN       NaN       NaN
...                     ...       ...       ...
2022-08-19 688799 -1.668211 -1.728929 -1.880395
           688800 -8.240506  0.124309 -0.288858
           688819 -2.045210  2.431337  1.111111
           688981 -2.125119 -1.914334 -1.537353
           689009 -3.770063 -1.790476 -3.805970

[18480 rows x 3 columns]

In [128]:
ind_base_data.xs(('2022-08-19','600519'))

zf1   -0.025851
zf2   -1.198644
zf3   -1.190402
Name: (2022-08-19 00:00:00, 600519), dtype: float64

In [234]:
start_time = datetime.datetime.now()
test = stk_day_all.close.groupby(level=1).apply(lambda x:x.pct_change())
end_time = datetime.datetime.now()
print(end_time - start_time)

0:00:02.475387


In [242]:
# %time stk_day_all.close.groupby(level=1).apply(lambda x:x.pct_change())

Wall time: 2.81 s


date        code  
2021-02-18  000001         NaN
            000002         NaN
            000004         NaN
            000005         NaN
            000006         NaN
                        ...   
2021-03-15  688777   -0.053763
            688788   -0.033625
            688819   -0.024326
            688981   -0.034304
            689009   -0.046135
Name: close, Length: 71504, dtype: float64

In [238]:
start_time = datetime.datetime.now()
tt = stk_day_all.groupby(level=1)['close'].pct_change()
end_time = datetime.datetime.now()
print(end_time - start_time)

0:00:00.021941


In [246]:
%time stk_day_all.groupby(level=1)['close'].pct_change(3)

Wall time: 19.9 ms


date        code  
2021-02-18  000001         NaN
            000002         NaN
            000004         NaN
            000005         NaN
            000006         NaN
                        ...   
2021-03-15  688777   -0.057963
            688788   -0.036735
            688819    0.014354
            688981    0.017710
            689009   -0.010880
Name: close, Length: 71504, dtype: float64

## 传入整个Datastruct

In [129]:
#多日涨幅
def drzf(data):
    return pd.DataFrame(zip(data.groupby(level=1)['close'].pct_change(1),\
                            data.groupby(level=1)['close'].pct_change(2),
                           data.groupby(level=1)['close'].pct_change(3),
                           data.groupby(level=1)['close'].pct_change(4),
                           data.groupby(level=1)['close'].pct_change(5),
                           data.groupby(level=1)['close'].pct_change(6),
                           data.groupby(level=1)['close'].pct_change(7),
                           ),index=data.index)

In [131]:
%time grp_method = stk_day_all.apply(lambda x:drzf(x))
# delta=26 192 ms
# delta = 150  1.08 s

Wall time: 33 ms


In [133]:
grp_method.xs(('2022-08-19','600519'))

0   -0.000259
1   -0.011986
2   -0.011904
Name: (2022-08-19 00:00:00, 600519), dtype: float64

## Using QA MA

In [317]:
def MAs(data):
    return pd.DataFrame(zip(data.close,
                            QA.MA(data.close,5),
                           QA.MA(data.close,10),
                           QA.MA(data.close,15),
                           QA.MA(data.close,20),
                           QA.MA(data.close,30),
                           QA.MA(data.close,40),
                           QA.MA(data.close,60),
                           QA.MA(data.close,80),
                           QA.MA(data.amount,5),
                           QA.MA(data.amount,20),
                           QA.MA(data.amount,120)),index=data.index)
#     return pd.DataFrame(QA.MA(data.close,5))

In [319]:
%time stk_day_all.add_func(MAs)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


Wall time: 13 s


0        1        2           3         4           5   \
date       code                                                                 
2020-10-19 000001   17.48      NaN      NaN         NaN       NaN         NaN   
           000002   27.74      NaN      NaN         NaN       NaN         NaN   
           000004   25.85      NaN      NaN         NaN       NaN         NaN   
           000005    2.67      NaN      NaN         NaN       NaN         NaN   
           000006    6.22      NaN      NaN         NaN       NaN         NaN   
...                   ...      ...      ...         ...       ...         ...   
2021-03-15 688777   72.16   75.736   81.637   82.487333   83.0140   83.613000   
           688788  101.74  105.102  109.967  114.175333  116.8510  117.483333   
           688819   44.52   44.424   44.801   44.930667   45.3595   46.893667   
           688981   55.74   56.120   57.225   57.080000   56.8880   56.438000   
           689009   61.82   62.850   66.752   69.440000   75.2565   83.445667   

                          6           7           8             9   \
date       code                                                      
2020-10-19 000001        NaN         NaN         NaN           NaN   
           000002        NaN         NaN         NaN           NaN   
           000004        NaN         NaN         NaN           NaN   
           000005        NaN         NaN         NaN           NaN   
           000006        NaN         NaN         NaN           NaN   
...                      ...         ...         ...           ...   
2021-03-15 688777   87.73375   91.996500         NaN  1.113700e+08   
           688788  118.75625  133.861667  155.719250  4.512277e+07   
           688819        NaN         NaN         NaN  1.979900e+08   
           688981   57.53575   57.303000   58.069625  1.426625e+09   
           689009   84.34600   82.039167   79.906125  1.445535e+08   

                             10  11  
date       code                      
2020-10-19 000001           NaN NaN  
           000002           NaN NaN  
           000004           NaN NaN  
           000005           NaN NaN  
           000006           NaN NaN  
...                         ...  ..  
2021-03-15 688777  1.427675e+08 NaN  
           688788  6.185034e+07 NaN  
           688819  2.593116e+08 NaN  
           688981  1.764530e+09 NaN  
           689009  2.214262e+08 NaN  

[390167 rows x 12 columns]

In [301]:
%time stk_day_all.close.groupby(level=1).apply(lambda x:x.rolling(5).mean())

Wall time: 3.49 s


date        code  
2020-10-19  000001        NaN
            000002        NaN
            000004        NaN
            000005        NaN
            000006        NaN
                       ...   
2021-03-15  688777     75.736
            688788    105.102
            688819     44.424
            688981     56.120
            689009     62.850
Name: close, Length: 390167, dtype: float64

In [305]:
%time stk_day_all.groupby(level=1)['close'].apply(lambda x:x.rolling(60).mean())

Wall time: 2.82 s


date        code  
2020-10-19  000001           NaN
            000002           NaN
            000004           NaN
            000005           NaN
            000006           NaN
                         ...    
2021-03-15  688777     91.996500
            688788    133.861667
            688819           NaN
            688981     57.303000
            689009     82.039167
Name: close, Length: 390167, dtype: float64

In [311]:
# QA vs min
# 6.76 s vs 13.7 s
# def MAs(data):
#     return pd.DataFrame(zip(data.groupby(level=1)['close'].apply(lambda x:x.rolling(5).mean()),
#                            data.groupby(level=1)['close'].apply(lambda x:x.rolling(10).mean()),
#                            data.groupby(level=1)['close'].apply(lambda x:x.rolling(15).mean()),
#                            data.groupby(level=1)['close'].apply(lambda x:x.rolling(20).mean())),index=data.index)

In [312]:
# %time stk_day_all.apply(lambda x:MAs(x))

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


Wall time: 13.7 s


0        1           2         3
date       code                                          
2020-10-19 000001      NaN      NaN         NaN       NaN
           000002      NaN      NaN         NaN       NaN
           000004      NaN      NaN         NaN       NaN
           000005      NaN      NaN         NaN       NaN
           000006      NaN      NaN         NaN       NaN
...                    ...      ...         ...       ...
2021-03-15 688777   75.736   81.637   82.487333   83.0140
           688788  105.102  109.967  114.175333  116.8510
           688819   44.424   44.801   44.930667   45.3595
           688981   56.120   57.225   57.080000   56.8880
           689009   62.850   66.752   69.440000   75.2565

[390167 rows x 4 columns]

In [282]:
def MAs(data):
    return data.rolling(5).mean(),data.rolling(10).mean(),data.rolling(15).mean()

In [283]:
# %time stk_day_all.groupby(level=1)['close'].apply(lambda x:x.rolling(5).mean())  #1.96 s
%time stk_day_all.groupby(level=1)['close'].apply(lambda x:MAs(x))  

Wall time: 2.19 s


code
000001    ([nan, nan, nan, nan, 22.924, 22.482, 21.988, ...
000002    ([nan, nan, nan, nan, 29.916000000000004, 30.5...
000004    ([nan, nan, nan, nan, 18.682, 18.848, 18.91399...
000005    ([nan, nan, nan, nan, 2.084, 2.112, 2.13000000...
000006    ([nan, nan, nan, nan, 5.1240000000000006, 5.20...
                                ...                        
688676    ([nan, nan, nan, nan, 20.014000000000003], [na...
605122    ([nan, nan, nan, nan], [nan, nan, nan, nan], [...
300952    ([nan, nan, nan], [nan, nan, nan], [nan, nan, ...
688083    ([nan, nan, nan], [nan, nan, nan], [nan, nan, ...
688667                 ([nan, nan], [nan, nan], [nan, nan])
Name: close, Length: 3994, dtype: object

## 传入groupby dataset

In [255]:
def drzf(data):
    return pd.DataFrame(zip(*(data.pct_change(i) for i in range(10)[1:])))

In [257]:
%time stk_day_all.groupby(level=1)['close'].apply(lambda x:drzf(x))

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


Wall time: 15.1 s


,0,1,2,3,4,5,6,7,8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.018519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.061635,-0.079012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.020107,-0.080503,-0.097531,NaN,NaN,NaN,NaN,NaN,NaN
4,0.010488,-0.009830,-0.070860,-0.088066,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.097800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.109310,-0.022200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
ind_base_data.loc['2021-03-05'].max(axis=1).sort_values(ascending=False).head(20)

date        code  
2021-03-05  002647    102.551020
            603222     84.362934
            603126     77.454545
            300234     73.076923
            300312     66.190476
            688059     65.456013
            601038     63.606911
            600010     57.575758
            002658     55.787476
            688308     54.871795
            601127     54.075547
            300293     44.675325
            600295     43.646409
            300022     41.646489
            002378     41.085271
            600714     40.892193
            002636     39.610390
            603032     37.692308
            300722     34.524089
            002554     33.484163
dtype: float64

In [168]:
stk_day_all.apply(lambda x:f(x))

zf1         zf2         zf3         zf4         zf5  \
date       code                                                                 
2021-02-18 000001         NaN         NaN         NaN         NaN         NaN   
           000002   23.497942         NaN         NaN         NaN         NaN   
           000004  -40.086638  -26.008230         NaN         NaN         NaN   
           000005  -88.932147  -93.368877  -91.810700         NaN         NaN   
           000006  150.251256  -72.302558  -83.405531  -79.506173         NaN   
...                       ...         ...         ...         ...         ...   
2021-03-15 688777  307.453416  -83.784270  217.745487  181.654957  -43.975155   
           688788   40.992239  474.477696  -77.137079  347.996477  297.111632   
           688819  -56.241400  -38.303769  151.383399  -89.995506   96.036988   
           688981   25.202156  -45.213289  -22.754989  214.737436  -87.474157   
           689009   10.907786   38.858940  -39.237271  -14.329268  249.068323   

                          zf6         zf7         zf8         zf9  
date       code                                                    
2021-02-18 000001         NaN         NaN         NaN         NaN  
           000002         NaN         NaN         NaN         NaN  
           000004         NaN         NaN         NaN         NaN  
           000005         NaN         NaN         NaN         NaN  
           000006         NaN         NaN         NaN         NaN  
...                       ...         ...         ...         ...  
2021-03-15 688777  378.197482  269.293756   -1.823129  300.666297  
           688788  -21.009317  574.221339  420.675537   38.421769  
           688819   73.770492  -65.434783  195.029821  127.840328  
           688981  145.442536  117.564403  -56.723602  269.383698  
           689009  -86.107865  172.214883  141.295863  -52.003106  

[71504 rows x 9 columns]

In [23]:
def drzf(data):
    temp = {
        'open':data.open,
        'high':data.high,
        'low': data.low,
        'close': data.close,
        'volume': data.volume,
        'amount':data.amount
    }
    #多日涨幅  统计过去1，2，3，4，5，6，7，8，9天的涨幅
#     drzf = {'zf{}'.format(i): data.close.pct_change(i)*100 for i in range(10)[1:10]}
    drzf = {'zf1': data.close.pct_change(1)*100 }
    temp.update(drzf)
    return pd.DataFrame(temp)

## 低效率

In [12]:
%%timeit
stk_day_all = QA.QA_fetch_stock_day_adv(code_list,start,end)
ind_base_data = stk_day_all.add_func(drzf)
#涨幅最大的20只
ind_base_data.loc['2021-03-05'].max(axis=1).sort_values(ascending=False).head(20)

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


44.1 s ± 564 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
start_time = datetime.datetime.now()
stk_day_all = QA.QA_fetch_stock_day_adv(code_list,start,end)
ind_base_data = stk_day_all.add_func(drzf)
#涨幅最大的20只
tt = ind_base_data.loc['2021-03-05'].max(axis=1).sort_values(ascending=False).head(20)
end_time = datetime.datetime.now()
end_time - start_time

d:\ProgramData\Anaconda3\lib\site-packages\QUANTAXIS\QAData\base_datastruct.py:841: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  return self.data.groupby(


datetime.timedelta(seconds=45, microseconds=140500)

In [28]:
stk_day_all.data

open    high     low   close     volume        amount
date       code                                                           
2021-02-18 000001   24.00   24.55   23.48   24.30  1505230.0  3.619388e+09
           000002   28.95   30.13   28.95   30.01   996922.0  2.956989e+09
           000004   17.21   18.07   17.21   17.98    23204.0  4.136155e+07
           000005    1.91    1.99    1.91    1.99    64388.0  1.259473e+07
           000006    4.88    5.02    4.88    4.98    70781.0  3.511906e+07
...                   ...     ...     ...     ...        ...           ...
2021-03-15 688777   75.80   75.80   71.00   72.16    17443.0  1.268197e+08
           688788  105.20  105.79  101.17  101.74     4391.0  4.510496e+07
           688819   45.81   45.83   44.30   44.52    43681.0  1.958270e+08
           688981   57.20   57.20   55.40   55.74   226351.0  1.272870e+09
           689009   65.78   66.88   60.51   61.82    22240.0  1.428158e+08

[71504 rows x 6 columns]

# 使用Numba加速

In [106]:
# @jit(nopython=True)
# def f(sn):
#     return pd.DataFrame(sn.pct_change(),index=sn.index,columns=['zf1'])
# #     return pd.DataFrame(zip(sn.pct_change(i) for i in range(10)))

In [107]:
# start_time = datetime.datetime.now()
# stk_day_all.apply(lambda x:f(x.close))
# end_time = datetime.datetime.now()
# print(end_time - start_time)

# 生产基础数据

## 多日涨幅

In [181]:
# sn.pct_change(1),sn.pct_change(2),sn.pct_change(3),sn.pct_change(4),sn.pct_change(5),sn.pct_change(6)
def drzf(sn):
    return pd.DataFrame(zip(*(sn.pct_change(n) for n in range(7)[1:])),index=sn.index,columns=['zf{}'.format(i) for i in range(7)[1:]])
#     return pd.DataFrame(zip(sn.pct_change(1),sn.pct_change(2),sn.pct_change(3),sn.pct_change(4),sn.pct_change(5),sn.pct_change(6)),index=sn.index,columns=['zf{}'.format(i) for i in range(7)[1:]])
#     return pd.DataFrame(zip(sn.pct_change(i) for i in range(10)))

In [182]:
# base_data = 
stk_day_all.apply(lambda x:drzf(x.close))

zf1       zf2       zf3       zf4       zf5       zf6
date       code                                                              
2021-02-18 000001       NaN       NaN       NaN       NaN       NaN       NaN
           000002  0.234979       NaN       NaN       NaN       NaN       NaN
           000004 -0.400866 -0.260082       NaN       NaN       NaN       NaN
           000005 -0.889321 -0.933689 -0.918107       NaN       NaN       NaN
           000006  1.502513 -0.723026 -0.834055 -0.795062       NaN       NaN
...                     ...       ...       ...       ...       ...       ...
2021-03-15 688777  3.074534 -0.837843  2.177455  1.816550 -0.439752  3.781975
           688788  0.409922  4.744777 -0.771371  3.479965  2.971116 -0.210093
           688819 -0.562414 -0.383038  1.513834 -0.899955  0.960370  0.737705
           688981  0.252022 -0.452133 -0.227550  2.147374 -0.874742  1.454425
           689009  0.109078  0.388589 -0.392373 -0.143293  2.490683 -0.861079

[71504 rows x 6 columns]

In [176]:
base_data.loc['2021-03-05'].max(axis=1).sort_values(ascending=False).head(20)

date        code  
2021-03-05  600519    729.535461
            601888    172.795455
            600763    113.661836
            600809    108.045614
            000661    107.809278
            688169     90.762115
            000596     77.760870
            000858     68.041451
            600436     67.665854
            600309     64.742515
            000568     62.555556
            002371     59.660000
            000799     59.588496
            002594     53.547222
            603236     51.452026
            600745     49.308057
            300751     48.123768
            002304     47.624324
            601799     46.415617
            002352     46.365854
dtype: float64

In [145]:
# performance checking
start_time = datetime.datetime.now()
stk_day_all.apply(lambda x:drzf(x.close))
end_time = datetime.datetime.now()
print(end_time - start_time)
# 00.179523

0:00:00.181549


# emas

In [212]:
def emas(data):
    return pd.DataFrame({'ema5':QA.EMA(data.close,5)})
#多日emas
#     return pd.DataFrame({'ema{}'.format(n): QA.EMA(data.close,n) for n in [5,10,15,20,30,40,60,80,100,120,240]})

In [206]:
stk_day_all.close

date        code  
2021-02-18  000001     24.30
            000002     30.01
            000004     17.98
            000005      1.99
            000006      4.98
                       ...  
2021-03-15  688777     72.16
            688788    101.74
            688819     44.52
            688981     55.74
            689009     61.82
Name: close, Length: 71504, dtype: float64

In [151]:
start_time = datetime.datetime.now()
test = stk_day_all.apply(lambda x:emas(x))
end_time = datetime.datetime.now()
print(end_time - start_time)

0:00:00.288775


In [213]:
stk_day_all.apply(lambda x:emas(x)).loc['2021-03-15	']


ema5
date       code              
2021-03-15 000001   55.480510
           000002   47.467007
           000004   37.331338
           000005   25.644225
           000006   18.859483
...                       ...
           688777  106.288514
           688788  104.772343
           688819   84.688229
           688981   75.038819
           689009   70.632546

[3902 rows x 1 columns]

## 平均涨幅

In [178]:
def mazf(data):
#多日波动幅度
    return pd.DataFrame({'mazf{}'.format(n): QA.MA(QA.ABS(data.close.pct_change(1)*100),n) for n in [5,10,15,20,30,40,60,80,100,120,240]}) 

In [163]:
start_time = datetime.datetime.now()
stk_day_all.apply(lambda x:mazf(x))
end_time = datetime.datetime.now()
print(end_time - start_time)

0:00:00.170544


In [179]:
stk_day_all.apply(lambda x:mazf(x))

mazf5      mazf10      mazf15      mazf20      mazf30  \
date       code                                                                 
2021-02-18 000001         NaN         NaN         NaN         NaN         NaN   
           000002         NaN         NaN         NaN         NaN         NaN   
           000004         NaN         NaN         NaN         NaN         NaN   
           000005         NaN         NaN         NaN         NaN         NaN   
           000006         NaN         NaN         NaN         NaN         NaN   
...                       ...         ...         ...         ...         ...   
2021-03-15 688777  470.885972  356.509933  265.538084  214.971227  215.183404   
           688788  463.062681  355.326287  263.730399  215.569368  216.056313   
           688819  472.039298  330.139766  258.073513  214.523949  216.363215   
           688981  105.181887  325.318485  254.049152  213.683563  215.722985   
           689009   88.159400  324.131884  249.777147  213.413205  207.075332   

                       mazf40      mazf60      mazf80     mazf100     mazf120  \
date       code                                                                 
2021-02-18 000001         NaN         NaN         NaN         NaN         NaN   
           000002         NaN         NaN         NaN         NaN         NaN   
           000004         NaN         NaN         NaN         NaN         NaN   
           000005         NaN         NaN         NaN         NaN         NaN   
           000006         NaN         NaN         NaN         NaN         NaN   
...                       ...         ...         ...         ...         ...   
2021-03-15 688777  199.512431  178.635882  164.573926  159.267334  148.771692   
           688788  199.391403  177.868521  164.237673  159.474608  147.761085   
           688819  199.844264  177.886077  164.037655  158.074120  147.680841   
           688981  199.037866  176.970833  161.589607  156.485738  145.162968   
           689009  197.992355  175.169027  160.994926  155.837792  144.626336   

                      mazf240  
date       code                
2021-02-18 000001         NaN  
           000002         NaN  
           000004         NaN  
           000005         NaN  
           000006         NaN  
...                       ...  
2021-03-15 688777  173.077941  
           688788  173.122843  
           688819  172.009633  
           688981  172.034381  
           689009  171.826968  

[71504 rows x 11 columns]

In [177]:
# ind_base_data = stk_day_all.add_func(drzf)

In [130]:
#涨幅最大的20只
ind_base_data.loc['2021-03-05'].max(axis=1).sort_values(ascending=False).head(20)

date        code  
2021-03-05  002647    102.551020
            603222     84.362934
            603126     77.454545
            300234     73.076923
            300312     66.190476
            688059     65.456013
            601038     63.606911
            600010     57.575758
            002658     55.787476
            688308     54.871795
dtype: float64

In [131]:
# 跌幅最大的20只
ind_base_data.loc['2021-03-05'].min(axis=1).sort_values(ascending=True).head(20)

date        code  
2021-03-05  300948   -36.629934
            300945   -36.611374
            300949   -33.256881
            603799   -32.865787
            600763   -31.932894
            688619   -31.670504
            003038   -29.525399
            605133   -29.063205
            300595   -27.415144
            300951   -26.923077
            605077   -26.686707
            300896   -26.536313
            688179   -26.345756
            603338   -25.854786
            600438   -25.816733
            002597   -25.450902
            601015   -25.306893
            002460   -25.089360
            600989   -24.926829
            601899   -24.812030
dtype: float64

In [202]:
sample = QA.QA_fetch_stock_day_adv(['000001','000002'],start,end).to_qfq()
# sample = QA.QA_fetch_stock_day_adv('000001',start,end).to_qfq()
sample.close.pct_change()

date        code  
2021-02-18  000001         NaN
            000002    0.234979
2021-02-19  000001   -0.205265
            000002    0.251153
2021-02-22  000001   -0.250000
            000002    0.328418
2021-02-23  000001   -0.262361
            000002    0.368445
2021-02-24  000001   -0.261579
            000002    0.353339
2021-02-25  000001   -0.263421
            000002    0.493436
2021-02-26  000001   -0.351925
            000002    0.548176
2021-03-01  000001   -0.351964
            000002    0.554779
2021-03-02  000001   -0.350825
            000002    0.537644
2021-03-03  000001   -0.308801
            000002    0.455889
2021-03-04  000001   -0.315821
            000002    0.426265
2021-03-05  000001   -0.316611
            000002    0.411817
2021-03-08  000001   -0.315155
            000002    0.444444
2021-03-09  000001   -0.331731
            000002    0.487770
2021-03-10  000001   -0.342360
            000002    0.504412
2021-03-11  000001   -0.306940
            000002  

QUANTAXIS.QAData.QADataStruct.QA_DataStruct_Stock_day

# 选择板块

In [83]:
QA.

SyntaxError: invalid syntax (<ipython-input-83-3708913aa175>, line 1)

In [17]:
import QUANTAXIS as QA

#获取全市场股票 list格式
code  = QA.QA_fetch_stock_list_adv().code.tolist()

# 获取全市场数据 QADataStruct格式

data =  QA.QA_fetch_stock_day_adv(code,  '2019-11-01','2020-01-06')

# 写个指标 so easy
def ifup20(data):
    return (data.close-QA.MA(data.close, 20)).dropna() > 0

# apply到 QADataStruct上

ind = data.add_func(ifup20)

# 对于指标groupby 日期 求和

ind.dropna().groupby(level=0).sum()

date
2019-11-28     978.0
2019-11-29    1031.0
2019-12-02    1101.0
2019-12-03    1423.0
2019-12-04    1583.0
2019-12-05    2164.0
2019-12-06    2508.0
2019-12-09    2618.0
2019-12-10    2651.0
2019-12-11    2558.0
2019-12-12    2272.0
2019-12-13    2809.0
2019-12-16    3228.0
2019-12-17    3446.0
2019-12-18    3452.0
2019-12-19    3476.0
2019-12-20    3353.0
2019-12-23    2366.0
2019-12-24    2933.0
2019-12-25    2867.0
2019-12-26    3021.0
2019-12-27    2774.0
2019-12-30    2716.0
2019-12-31    2861.0
2020-01-02    3271.0
2020-01-03    3298.0
2020-01-06    3266.0
Name: close, dtype: float64

In [34]:
t=ind['2020-01-06']
t[t==True]

code
000001    True
000002    True
000004    True
000005    True
000006    True
000008    True
000009    True
000010    True
000011    True
000012    True
000014    True
000016    True
000017    True
000019    True
000020    True
000021    True
000023    True
000025    True
000026    True
000027    True
000028    True
000030    True
000031    True
000032    True
000036    True
000040    True
000042    True
000045    True
000046    True
000048    True
          ... 
688023    True
688128    True
688199    True
688202    True
688389    True
300800    True
688021    True
688288    True
688299    True
688363    True
300080    True
300749    True
688166    True
002453    True
002542    True
300796    True
688101    True
688300    True
300803    True
688138    True
300805    True
688196    True
601916    True
688310    True
688358    True
002323    True
688357    True
688399    True
300810    True
688198    True
Name: close, Length: 3266, dtype: bool

In [1]:
#!/usr/bin/env python3

import datetime
import os

import pandas as pd

import QUANTAXIS as QA

# code_list = QA.QA_fetch_stock_list().index.tolist()
code_list = ["000001", "600000"]
current_date = str(datetime.date.today())
current_daily_bar = pd.DataFrame()
# 前置数据时间点
start_date = QA.QA_util_get_pre_trade_date(current_date, n=100)
# 前置数据准备
daily_data = QA.QA_fetch_stock_day_adv(
    code=code_list, start=start_date, end=current_date
).to_qfq()

# 方便计算，数据转换为相应透视数据
closes = daily_data.pivot("close")
highs = daily_data.pivot("high")
lows = daily_data.pivot("low")


def get_short_strength(
    highs: pd.DataFrame, lows: pd.DataFrame, closes: pd.DataFrame, period: int = 27
) -> pd.Series:
    """
    计算空方力量

    :param highs: high 透视数据
    :param closes: close 透视数据
    :param period: 周期
    """
    A = highs.apply(QA.HHV, N=period, axis=0)
    B = lows.apply(QA.LLV, N=period, axis=0)
    return 100 * (A.iloc[-1] - closes.ffill().iloc[-1]) / (A.iloc[-1] - B.iloc[-1])


def get_long_strength(
    highs: pd.DataFrame, lows: pd.DataFrame, closes: pd.DataFrame, period: int = 27
) -> pd.Series:
    """
    计算多方力量
    """
    A = highs.apply(QA.HHV, N=period, axis=0)
    B = lows.apply(QA.LLV, N=period, axis=0)
    C = ((closes - B) / (A - B)).apply(QA.QAIndicator.base.SMA, N=5, M=1, axis=0) * 100
    return (3 * C - 2 * C.apply(QA.QAIndicator.base.SMA, N=3, M=1)).iloc[-1]


def get_RSV(
    highs: pd.DataFrame, lows: pd.DataFrame, closes: pd.DataFrame, period: int = 30
) -> pd.DataFrame:
    """
    计算 RSV 值
    """
    return (
        100
        * (closes - lows.apply(QA.LLV, N=period, axis=0))
        / (highs.apply(QA.HHV, N=period, axis=0) - lows.apply(QA.LLV, N=period, axis=0))
    )


def get_KDJ(
    highs: pd.DataFrame,
    lows: pd.DataFrame,
    closes: pd.DataFrame,
    period: int = 30,
    M1: int = 3,
    M2: int = 3,
) -> dict:
    """
    获取 KDJ 值
    """
    RSV = get_RSV(highs, lows, closes, period)
    KDJ_K = RSV.apply(QA.QAIndicator.base.SMA, N=3, M=1, axis=0)
    KDJ_D = KDJ_K.apply(QA.QAIndicator.base.SMA, N=3, M=1, axis=0)
    KDJ_J = 3 * KDJ_K - 2 * KDJ_D
    return {"KDJ_K": KDJ_K, "KDJ_D": KDJ_D, "KDJ_J": KDJ_J}